In [1]:
import pyrebase
import uuid

from core.config import db

problems = db.child("user").get()

# Iterate through the retrieved data
for problem in problems.each():
    print(problem.key(), problem.val())

-OCtVr9MP27UrtIN44zw {'cart': ['book1', 'book2'], 'date': '2024-11-29', 'gender': True, 'history': ['order1', 'order2'], 'id': '1', 'info': 'System Administrator', 'managed_users': ['2', '3', '4'], 'name': 'John Doe', 'password': 'securepassword', 'permissions': ['manage_users', 'view_reports', 'edit_content'], 'phone': '0123456789', 'role': 'Admin', 'username': 'admin123'}
-OCtgV62_YOYSYcme8Qs {'date': '2024-11-29', 'gender': True, 'history': ['delivery1', 'delivery2'], 'id': '3', 'info': 'Delivery Specialist', 'name': 'Mike Johnson', 'password': 'deliversecure', 'phone': '0123456789', 'region': 'Downtown', 'role': 'DeliveryMan', 'username': 'delivery_mike'}


In [2]:
from models.users import  create_user

admin_data = {
    "id": "1",
    "name": "John Doe",
    "username": "admin123",
    "password": "securepassword",
    "date": "2024-11-29",
    "phone": "0123456789",
    "gender": True,
    "info": "System Administrator",
    "role": "Admin",
    "permissions": ["manage_users", "view_reports", "edit_content"],
    "managed_users": ["2", "3", "4"],
}

# Dữ liệu mẫu cho RegularUser
user_data = {
    "id": "2",
    "name": "Jane Smith",
    "username": "jane_smith",
    "password": "userpassword",
    "date": "2024-11-29",
    "phone": "0123456789",
    "gender": False,
    "info": "Regular customer",
    "role": "User",
    "cart": ["book1", "book2"],
    "history": ["order1", "order2"],
    # "subscription_type": "Premium",
    # "wishlist": ["book3", "book4"],
}

# Dữ liệu mẫu cho DeliveryMan
delivery_man_data = {
    "id": "3",
    "name": "Mike Johnson",
    "username": "delivery_mike",
    "password": "deliversecure",
    "date": "2024-11-29",
    "phone": "0123456789",
    "gender": True,
    "info": "Delivery Specialist",
    "role": "DeliveryMan",
    "history": ["delivery1", "delivery2"],
    # "vehicle": "Motorbike",
    "region": "Downtown",
    # "completed_deliveries": 120,
    # "rating":[5, 5]
}

# Tạo đối tượng bằng Factory Method
admin = create_user(admin_data)
user1 = create_user(user_data)
delivery_man = create_user(delivery_man_data)

# Kiểm tra kết quả
print(admin)
print(user1)
print(delivery_man)

id='1' name='John Doe' username='admin123' password='securepassword' date='2024-11-29' gender=True info='System Administrator' phone='0123456789' role='Admin' permissions=['manage_users', 'view_reports', 'edit_content'] managed_users=['2', '3', '4']
id='2' name='Jane Smith' username='jane_smith' password='userpassword' date='2024-11-29' gender=False info='Regular customer' phone='0123456789' role='User' cart=['book1', 'book2'] history=['order1', 'order2']
id='3' name='Mike Johnson' username='delivery_mike' password='deliversecure' date='2024-11-29' gender=True info='Delivery Specialist' phone='0123456789' role='DeliveryMan' history=['delivery1', 'delivery2'] region='Downtown'


In [ ]:
# # Chuyển đối tượng RegularUser thành dictionary
# user_dict = user1.dict()

# # Đẩy dữ liệu lên Firebase
# db.child("user").push(user_dict)


{'name': '-OCtHDbLiMdhCo830cW9'}

In [ ]:
from repositories.db_repository import FirebaseService
from models.users import User, create_user
from collections import OrderedDict


class UserRepository:
    def __init__(self):
        self.collection_name = "user"

    def add_user(self, user_data: dict):
        # Tạo đối tượng User dựa trên role
        user = create_user(user_data)
        key = user.id
        FirebaseService.add_data(self.collection_name, key, user.dict())
        return {"message": "User created successfully."}

    def get_all_users(self):
        # Lấy tất cả người dùng từ Firebase
        users_data = FirebaseService.get_data(self.collection_name)
        # return [create_user(user) for user in users_data]
        if isinstance(users_data, OrderedDict):
        # Duyệt qua các giá trị trong OrderedDict và tạo đối tượng User
            return [create_user(user_data) for user_data in users_data.values()]
        else:
            raise ValueError("Data from Firebase is not in expected format")


    def get_user_by_id(self, user_id: str):
        users_data = FirebaseService.get_data_by_key(self.collection_name, user_id)
        # user_get = db.child(self.collection_name).order_by_child("id").equal_to(user_id).get()
        # # print(user_get)
        # for user in user_get.each():
        #     # print(user.key())  # In khóa của mỗi bản ghi
        #     # print(user.val()) 
        #     return create_user(user.val())
        return create_user(users_data.val())

    def update_user(self, user_id: str, user_data: dict):
        # Tạo đối tượng User và cập nhật dữ liệu
        usernew = create_user(user_data)
        # user_get = db.child(self.collection_name).order_by_child("id").equal_to(user_id).get()
        # # print(user_get)
        # for user in user_get.each():
        #     tmp_key = user.key()
        #     db.child("user").child(tmp_key).update(usernew.dict())
        FirebaseService.update_data(self.collection_name, user_id, usernew.dict())
        return {"message": "User updated successfully."}

    def delete_user(self, user_id: str):
        FirebaseService.delete_data(self.collection_name, user_id)
        return {"message": "User deleted successfully."}


# Khởi tạo một instance cho repository
user_repository = UserRepository()


In [4]:
user = create_user(user_data)
key = user.id
FirebaseService.add_data("user", key, user.dict())

In [5]:
user_repository.add_user(user_data)

{'message': 'User created successfully.'}

In [4]:
all_users = user_repository.get_all_users()
print(all_users)

[Admin(id='2', name='John Doe', username='admin123', password='securepassword', date='2024-11-29', gender=True, info='System Administrator', phone='0123456789', role='Admin', permissions=['manage_users', 'view_reports', 'edit_content'], managed_users=['2', '3', '4']), Admin(id='1', name='John Doe', username='admin123', password='securepassword', date='2024-11-29', gender=True, info='System Administrator', phone='0123456789', role='Admin', permissions=['manage_users', 'view_reports', 'edit_content'], managed_users=['2', '3', '4']), DeliveryMan(id='3', name='Mike Johnson', username='delivery_mike', password='deliversecure', date='2024-11-29', gender=True, info='Delivery Specialist', phone='0123456789', role='DeliveryMan', history=['delivery1', 'delivery2'], region='Downtown')]


In [5]:
all_users=user_repository.get_user_by_id( "1")
print(all_users)

id='1' name='John Doe' username='admin123' password='securepassword' date='2024-11-29' gender=True info='System Administrator' phone='0123456789' role='Admin' permissions=['manage_users', 'view_reports', 'edit_content'] managed_users=['2', '3', '4']


In [16]:
collection = "user"
key = "1"
user_get = db.child(collection).order_by_child("id").equal_to(key).get()
# print(user_get)
for user in user_get.each():
    print(user.key())  # In khóa của mỗi bản ghi
    print(user.val()) 
    create_user(user.val())

-OCtVr9MP27UrtIN44zw
{'date': '2024-11-29', 'gender': True, 'id': '1', 'info': 'System Administrator', 'managed_users': ['2', '3', '4'], 'name': 'John Doe', 'password': 'securepassword', 'permissions': ['manage_users', 'view_reports', 'edit_content'], 'phone': '0123456789', 'role': 'Admin', 'username': 'admin123'}


In [13]:
users_data1 = FirebaseService.get_data_by_key("user","2")
print(users_data1)

OrderedDict([('-OCtUuomHx8WrsFz8EnQ', {'date': '2024-11-29', 'gender': True, 'id': '2', 'info': 'System Administrator', 'managed_users': ['2', '3', '4'], 'name': 'John Doe', 'password': 'securepassword', 'permissions': ['manage_users', 'view_reports', 'edit_content'], 'phone': '0123456789', 'role': 'Admin', 'username': 'admin123'})])


In [13]:
user_repository.update_user("2",admin_data)

{'message': 'User updated successfully.'}

In [12]:
user_repository.update_user("1",user_data)

{'message': 'User updated successfully.'}

In [ ]:
FirebaseService.update_data("user", "1",user_data)

In [4]:
user_repository.delete_user("2")

{'message': 'User deleted successfully.'}